In [50]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
#  Y is a 1682x943 matrix, containing ratings (1-5) of 1682 movies on 
#  943 users
#  R is a 1682x943 matrix, where R(i,j) = 1 if and only if user j gave a
#  rating to movie i
data = sio.loadmat('ex8_movies.mat')

Y = data['Y'] #[1682*943] movies * users
R = data['R'] #[1682*943] 0 or 1
print Y.shape,R.shape
print Y[0:5,0:5]
print 'Average rating for movie 1 (Toy Story): %f / 5'\
      %(np.mean(Y[0,R[0,:]]))





(1682L, 943L) (1682L, 943L)


[[5 4 0 0 4]
 [3 0 0 0 3]
 [4 0 0 0 0]
 [3 0 0 0 0]
 [3 0 0 0 0]]
Average rating for movie 1 (Toy Story): 4.520679 / 5


In [51]:
#load movies data set
params = sio.loadmat('ex8_movieParams.mat')
X = params['X']
Theta = params['Theta']
num_users = params['num_users']
num_movies = params['num_movies']
num_features = params['num_features']
print X.shape,Theta.shape
print num_movies,num_users,num_features



(1682L, 10L) (943L, 10L)
[[1682]] [[943]] [[10]]


In [52]:
#compute the cost
#note:no x0 = 1, no 1/m  !!!!!!!!
def computeCost(x,theta,y,r,reg=0):
    j = 0
    x_grad = np.zeros_like(x)
    theta_grad = np.zeros_like(theta)
    
    j_temp = (x.dot(theta.T) - y)**2
    j = 0.5 * np.sum(j_temp[r==1]) + 0.5 * reg * np.sum(theta**2) \
        + 0.5 * reg * np.sum(x**2) 
    
    x_grad = np.dot(((x.dot(theta.T)-y)*r),theta) + reg * x
    theta_grad = np.dot(((x.dot(theta.T)-y)*r).T,x) + reg * theta
    
    return j,x_grad,theta_grad
    
#Reduce the data set size so that this runs faster
nu, nm, nf = 4, 5, 3
X1 = X[0:nm,0:nf]
Theta1 = Theta[0:nu,0:nf]
Y1 = Y[0:nm,0:nu]
R1 = R[0:nm,0:nu]

#check the cost
J,x_g,theta_g = computeCost(X1,Theta1,Y1,R1,0)
print"reg = 0,this value should be about 22.22\n",J

J2,x_g2,theta_g2 = computeCost(X1,Theta1,Y1,R1,1.5)
print"reg = 1.5,this value should be about 31.34\n",J2

reg = 0,this value should be about 22.22
22.2246037257
reg = 1.5,this value should be about 31.34
31.3440562443


In [53]:
#compute numerical_gradient
def eval_numerical_gradient(f, x, verbose=True, h=0.00001):
    """ 
    a naive implementation of numerical gradient of f at x 
    - f should be a function that takes a single argument
    - x is the point (numpy array) to evaluate the gradient at
    """ 
    
    fx = f(x) # evaluate function value at original point
    grad = np.zeros_like(x)
    # iterate over all indexes in x
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        
        # evaluate function at x+h
        ix = it.multi_index
        oldval = x[ix]
        x[ix] = oldval + h # increment by h
        fxph = f(x) # evalute f(x + h)
        x[ix] = oldval - h
        fxmh = f(x) # evaluate f(x - h)
        x[ix] = oldval # restore
        
        # compute the partial derivative with centered formula
        grad[ix] = (fxph - fxmh) / (2 * h) # the slope
        if verbose:
          print ix, grad[ix]
        it.iternext() # step to next dimension
    return grad

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))


In [44]:
#check the gradient 
#create the small data

x_t = np.random.rand(4,3)
theta_t = np.random.rand(5,3)
y_t = x_t.dot(theta_t.T) #4,5
y_t[np.random.rand(*y_t.shape)>0.5] = 0
r_t = np.zeros_like(y_t)
r_t[y_t != 0] = 1
#print y_t,r_t

X_t = np.random.randn(*x_t.shape)
Theta_t = np.random.randn(*theta_t.shape)
nu_t = 5
nm_t = 4
nf_t = 3

#j,x_grad,theta_grad = computeCost(X1,Theta1,Y1,R1,0)
#f = lambda W:computeCost(X1,Theta1,Y1,R1,0)[0]
j,x_grad,theta_grad = computeCost(X_t,Theta_t,y_t,r_t,1.5)
f = lambda W:computeCost(X_t,Theta_t,y_t,r_t,1.5)[0] #f = J(x)
x_grad1 = eval_numerical_gradient(f,X_t , verbose=False, h=0.00001)
theta_grad1 = eval_numerical_gradient(f, Theta_t, verbose=False, h=0.00001)
display_X = np.hstack((x_grad,x_grad1))
print display_X
display_Theta = np.hstack((theta_grad,theta_grad1))
print display_Theta

print '%s max relative error: %e' % ('x', rel_error(x_grad, x_grad1))
print '%s max relative error: %e' % \
      ('theta', rel_error(theta_grad, theta_grad1))



[[  0.14401176  -6.22590369  10.0516782    0.14401176  -6.22590369
   10.0516782 ]
 [  4.45625126   2.03223863   7.35329766   4.45625126   2.03223863
    7.35329766]
 [  0.11515446  -9.74467008  -1.91337529   0.11515446  -9.74467008
   -1.91337529]
 [  0.90141124  -1.28554704   1.55376368   0.90141124  -1.28554704
    1.55376368]]
[[ -0.12785057  -0.73651273  -1.21583757  -0.12785057  -0.73651273
   -1.21583757]
 [ -2.72597491   4.66380798   0.23347243  -2.72597491   4.66380798
    0.23347243]
 [ -0.42229439  -0.40927681 -11.5753487   -0.42229439  -0.40927681
  -11.5753487 ]
 [  3.61809669  12.26402186   7.18521682   3.61809669  12.26402186
    7.18521682]
 [  1.15495755  -1.53477221  -2.08059821   1.15495755  -1.53477221
   -2.08059821]]
x max relative error: 4.429443e-10
theta max relative error: 4.670187e-10


In [54]:
def load_data(filename):
    movieList = []
    file = open(filename)
    for line in file.readlines():
        lineArr = line.strip().split(' ')
        col_num = len(lineArr)
        temp = ''
        for i in xrange(col_num):
            if i==0:
                continue
            temp= temp + lineArr[i] + ' '
        movieList.append(temp)
    return movieList

movieList = load_data('movie_ids.txt')
print movieList[:5]


['Toy Story (1995) ', 'GoldenEye (1995) ', 'Four Rooms (1995) ', 'Get Shorty (1995) ', 'Copycat (1995) ']


In [55]:
#Entering ratings for a new user
my_ratings = np.zeros((1682,1))
my_ratings[0] = 4
my_ratings[97] = 2
my_ratings[6] = 3
my_ratings[11]= 5
my_ratings[53] = 4
my_ratings[63]= 5
my_ratings[65]= 3
my_ratings[68] = 5
my_ratings[182] = 4
my_ratings[225] = 5
my_ratings[354]= 5
for i in range(1682):
    if my_ratings[i]>0:
        print "Rated %d for %s" %(my_ratings[i],movieList[i])






Rated 4 for Toy Story (1995) 
Rated 3 for Twelve Monkeys (1995) 
Rated 5 for Usual Suspects, The (1995) 
Rated 4 for Outbreak (1995) 
Rated 5 for Shawshank Redemption, The (1994) 
Rated 3 for While You Were Sleeping (1995) 
Rated 5 for Forrest Gump (1994) 
Rated 2 for Silence of the Lambs, The (1991) 
Rated 4 for Alien (1979) 
Rated 5 for Die Hard 2 (1990) 
Rated 5 for Sphere (1998) 


In [60]:
#Learning Movie Ratings

#Add our own ratings to the data matrix
YY = np.hstack((my_ratings,Y))
RR = np.hstack((my_ratings!=0,R))
print YY.shape,RR.shape
#Normalize Ratings

def normalizeRatings(y,r):
    m,n = y.shape
    ymean = np.zeros((m,1))
    ynorm = np.zeros_like(y)
    for i in xrange(m):
        idx = np.where(r[i,:]==1)
        ymean[i] = np.mean(y[i,idx])
        ynorm[i,idx] = y[i,idx] - ymean[i]
        
    return ymean,ynorm
    
Ymean,Ynorm = normalizeRatings(YY,RR)

num_users1 = YY.shape[1]
num_movies1 = YY.shape[0]
num_features1 = 10
#Set Initial Parameters (Theta, X)
XX = np.random.randn(num_movies1,num_features1)
TTheta = np.random.randn(num_users1,num_features1)
print XX.shape
print TTheta.shape









(1682L, 944L) (1682L, 944L)
(1682L, 10L)
(944L, 10L)


In [69]:
from scipy import optimize

#we use the scipy.optimize.fmin_cg,so we need to change the 
#function computeCost(),beacause x must be 1-D

args = (Ynorm,RR,num_users1,num_movies1,num_features1,1.5)
params = np.hstack((XX.ravel(),TTheta.ravel())).ravel()

def Cost(params,*args):
    '''now params is 1-D,include [x,theta]'''
    y,r,nu,nm,nf,reg = args
    j = 0
    x = params[0:nm*nf].reshape(nm,nf)
    theta = params[nm*nf:].reshape(nu,nf)
    
    j_temp = (x.dot(theta.T) - y)**2
    j = 0.5 * np.sum(j_temp[r==1]) + 0.5 * reg * np.sum(theta**2) \
        + 0.5 * reg * np.sum(x**2) 
    
    return j

def grad(params,*args):
    y,r,nu,nm,nf,reg = args
    #unfold into x and theta
    x = params[0:nm*nf].reshape(nm,nf)
    theta = params[nm*nf:].reshape(nu,nf)
    x_grad = np.zeros_like(x)
    theta_grad = np.zeros_like(theta)
    x_grad = np.dot(((x.dot(theta.T)-y)*r),theta) + reg * x
    theta_grad = np.dot(((x.dot(theta.T)-y)*r).T,x) + reg * theta
    #fold the x_grad and theta_grad
    g = np.hstack((x_grad.ravel(),theta_grad.ravel())).ravel()
    return g

    
res = optimize.fmin_cg(Cost,x0=params,fprime=grad,args=args,maxiter=100)
#get the bestX,bestTheta

bestX = res[0:num_movies1*num_features1].reshape(num_movies1,num_features1)
bestTheta = res[num_movies1*num_features1:].reshape(num_users1,num_features1)


print bestX.shape,bestTheta.shape

score = bestX.dot(bestTheta.T) + Ymean

my_score = score[:,0]  #line 1 is my scoce
print score.shape
print my_score[:5]
sort_index = my_score.argsort()
favorite = 3
for i in xrange(favorite):
    print "favorite %d score is %d,for:%s" \
          %(i+1,my_score[sort_index[-(i+1)]],movieList[sort_index[-(i+1)]])

         Current function value: 27769.265138
         Iterations: 100
         Function evaluations: 152
         Gradient evaluations: 152
(1682L, 10L) (944L, 10L)
(1682L, 944L)
[ 3.89489124  3.56569854  3.25036093 ...,  1.99989186  2.99999858
  3.00004241]
favorite 1 score is 5,for:Return of the Jedi (1983) 
favorite 2 score is 5,for:Star Wars (1977) 
favorite 3 score is 5,for:Prefontaine (1997) 


In [49]:
#an example of optimize
args = (2, 3, 7, 8, 9, 10)  # parameter values
def f(x, *args):
    
    u, v = x
    a, b, c, d, e, f = args
    return a*u**2 + b*u*v + c*v**2 + d*u + e*v + f
def gradf(x, *args):
    u, v = x
    a, b, c, d, e, f = args
    gu = 2*a*u + b*v + d     # u-component of the gradient
    gv = b*u + 2*c*v + e     # v-component of the gradient
    return np.asarray((gu, gv))
x0 = np.asarray((0, 0))  # Initial guess.
from scipy import optimize
res1 = optimize.fmin_cg(f, x0, fprime=gradf, args=args)
print res1

Optimization terminated successfully.
         Current function value: 1.617021
         Iterations: 4
         Function evaluations: 8
         Gradient evaluations: 8
[-1.80851064 -0.25531915]


In [70]:
y = np.array([1,5,3,2,4,6])
yy = ['a','b','c','d','e','f']
index = np.argsort(y)
print index
for i in xrange(3):
    print "favorite %d score is %d,for %s" \
          %(i+1,y[index[-(i+1)]],yy[index[-(i+1)]])


[0 3 2 4 1 5]
favorite 1 score is 6,for f
favorite 2 score is 5,for b
favorite 3 score is 4,for e
